In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/

/content


In [ ]:
# Should be on /content to run this code
!pip install kaggle
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [ ]:
import json
token = {"username":"joeweiliang","key":"094d7d2959d315ea0a610fcf52daeca8"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
#These 3 lines below may need to run twice
!chmod 600 /content/.kaggle/kaggle.json

In [ ]:
!cp /content/.kaggle/kaggle.json /root/.kaggle/

In [ ]:
!kaggle config set -n path -v /content

- path is now set to: /content


In [ ]:
# Upgrade Kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

Processing /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303/kaggle-1.5.12-cp37-none-any.whl
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle competitions download -c 11785-spring2021-hw3p2

  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 36.8MB/s]
 98% 370M/377M [00:13<00:00, 30.2MB/s]
100% 377M/377M [00:13<00:00, 29.9MB/s]
  0% 0.00/81.3k [00:00<?, ?B/s]
100% 81.3k/81.3k [00:00<00:00, 73.9MB/s]
100% 3.52G/3.52G [01:29<00:00, 62.2MB/s]
100% 3.52G/3.52G [01:29<00:00, 42.3MB/s]
  0% 0.00/1.06k [00:00<?, ?B/s]
100% 1.06k/1.06k [00:00<00:00, 1.02MB/s]
  0% 0.00/338k [00:00<?, ?B/s]
100% 338k/338k [00:00<00:00, 108MB/s]
100% 310M/310M [00:09<00:00, 33.7MB/s]



In [ ]:
# TODO: Unzip the dataset (check where the dataset is downloaded)
#!unzip competitions/11785-spring2021-hw3p2/dev.npy.zip -d /content/drive/MyDrive/colab_notebooks/11785DL/HW3/
#!cp competitions/11785-spring2021-hw3p2/dev_labels.npy /content/drive/MyDrive/colab_notebooks/11785DL/HW3/
#!cp competitions/11785-spring2021-hw3p2/phoneme_list.py /content/drive/MyDrive/colab_notebooks/11785DL/HW3/
#!cp sample_submission.csv /content/drive/MyDrive/colab_notebooks/11785DL/HW3/

#!unzip competitions/11785-spring2021-hw3p2/test.npy.zip -d /content/drive/MyDrive/colab_notebooks/11785DL/HW3/
#!unzip competitions/11785-spring2021-hw3p2/train_labels.npy.zip -d /content/drive/MyDrive/colab_notebooks/11785DL/HW3/
#!unzip competitions/11785-spring2021-hw3p2/train.npy.zip -d /content/drive/MyDrive/colab_notebooks/11785DL/HW3/


Archive:  competitions/11785-spring2021-hw3p2/train.npy.zip
  inflating: /content/drive/MyDrive/colab_notebooks/11785DL/HW3/train.npy  


In [ ]:
%cd /content/drive/MyDrive/colab_notebooks/11785DL/HW3/

/content/drive/MyDrive/colab_notebooks/11785DL/HW3


In [ ]:
%ls

dev_labels.npy  phoneme_list.py        test.npy          train.npy
dev.npy         sample_submission.csv  train_labels.npy


In [ ]:
import numpy as np
train_data = np.load('train.npy', allow_pickle=True)
train_labels = np.load('train_labels.npy', allow_pickle=True)
dev_data = np.load('dev.npy', allow_pickle=True)
dev_labels = np.load('dev_labels.npy', allow_pickle=True)
test_data = np.load('test.npy', allow_pickle=True)

In [ ]:
cnt = 0
for i in range(test_data.shape[0]):
  cnt += test_data[i].shape[0]
cnt

1894499

In [ ]:
test_data[2].shape

(424, 40)

In [ ]:
# Import necessary libraries

import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.metrics import accuracy_score
import time
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Phoneme information
import phoneme_list as pl

In [ ]:
p_list = pl.PHONEME_LIST
p_map = pl.PHONEME_MAP
print(p_list)
print(p_map)

[' ', 'SIL', 'SPN', 'AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', 'H', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH']
[' ', '.', '!', 'a', 'A', 'h', 'o', 'w', 'y', 'b', 'c', 'd', 'D', 'e', 'r', 'E', 'f', 'g', 'H', 'i', 'I', 'j', 'k', 'l', 'm', 'n', 'N', 'O', 'Y', 'p', 'R', 's', 'S', 't', 'T', 'u', 'U', 'v', 'W', '?', 'z', 'Z']


In [ ]:
train_data.shape[0]

14542

In [ ]:
for i in range(10):
  print(train_data[i].shape)

(1184, 40)
(1083, 40)
(1316, 40)
(1435, 40)
(1535, 40)
(1376, 40)
(1309, 40)
(1312, 40)
(847, 40)
(1168, 40)


In [ ]:
for i in range(10):
  print(train_labels[i])

[1, 4, 25, 11, 4, 40, 33, 36, 18, 19, 40, 25, 3, 23, 19, 21, 5, 37, 12, 5, 31, 33, 15, 33, 19, 25, 38, 19, 10, 12, 5, 29, 30, 13, 24, 5, 31, 5, 40, 18, 4, 11, 9, 19, 25, 16, 7, 25, 11, 1, 4, 25, 11, 18, 19, 40, 18, 4, 37, 19, 26, 31, 27, 16, 5, 24, 19, 23, 39, 14, 23, 20, 30, 20, 23, 8, 11, 5, 29, 3, 25, 12, 5, 16, 4, 22, 33, 5, 37, 25, 27, 30, 3, 9, 14, 20, 18, 4, 37, 19, 26, 33, 15, 22, 5, 25, 29, 23, 15, 31, 1, 4, 40, 5, 25, 3, 30, 17, 39, 5, 24, 5, 25, 33, 6, 25, 18, 19, 40, 27, 25, 9, 19, 18, 4, 16, 1]
[1, 18, 20, 22, 5, 25, 33, 13, 25, 11, 19, 11, 12, 5, 33, 18, 20, 18, 4, 11, 18, 19, 24, 31, 13, 23, 16, 9, 19, 25, 5, 24, 5, 26, 12, 5, 22, 30, 7, 11, 12, 4, 33, 29, 35, 32, 33, 1, 19, 25, 33, 5, 12, 5, 18, 7, 31, 1, 5, 23, 6, 26, 38, 19, 12, 5, 24, 4, 21, 19, 31, 33, 30, 15, 33, 31, 1, 12, 4, 33, 16, 30, 5, 24, 18, 19, 40, 29, 30, 20, 37, 20, 5, 31, 5, 22, 38, 15, 25, 33, 5, 25, 31, 38, 19, 12, 5, 30, 36, 24, 40, 5, 25, 11, 12, 13, 30, 6, 30, 11, 5, 25, 13, 30, 20, 22, 5, 25, 11, 

In [ ]:
#train_labels[0]

word = ''
for i in train_labels[0]:
    word += p_map[i]

word

'.AndAztUHiznalijhvDhstEtinWicDhpRemhshzHAdbinfwnd.AndHizHAviNsOfhmil?rlIRIlydhpanDhfAkthvnORabrIHAviNtEkhnplEs.AzhnaRg?hmhntonHizOnbiHAf.'

### Preprocessing + Data Loader

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X  # (timestep, band)
        self.Y = Y  # label list

        assert(len(self.X) == len(self.Y))

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, index):
        data = self.X[index]
        label = self.Y[index]
        data = torch.Tensor(data)
        label = torch.IntTensor(label)
        return data, label

    def collate_fn(self, batch):
        # 1 training data, 1 label
        batch_x = [x for x,y in batch]
        batch_y = [y for x,y in batch]
        batch_x = nn.utils.rnn.pad_sequence(batch_x)
        batch_y = nn.utils.rnn.pad_sequence(batch_y, batch_first=True)
        batch_x_len = [len(x) for x,y in batch]
        batch_y_len = [len(y) for x,y in batch]
        batch_x_len = torch.IntTensor(batch_x_len)
        batch_y_len = torch.IntTensor(batch_y_len)

        #data_length = torch.IntTensor(data_length)
        #target_length = torch.IntTensor(target_length)

        return batch_x, batch_y, batch_x_len, batch_y_len
        ###############################################################
        # batch_x: (T,B,40)     timestep(longest in batch), batch, band
        # batch_y: (B,T)        batch, labels(longest in batch)
        # batch_x_len: []       list of each timestep in batch
        # batch_y_len: []       list of each label in batch

In [ ]:
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0 #change back to 8 when training for speed

#Training
train_dataset = MyDataset(train_data, train_labels)
train_loader_args = dict(shuffle=True, batch_size=64, num_workers=num_workers, pin_memory=True, collate_fn=train_dataset.collate_fn) if cuda \
                    else dict(shuffle=True, batch_size=64)
train_loader = DataLoader(train_dataset, **train_loader_args)

#Validation
dev_dataset = MyDataset(dev_data, dev_labels)
dev_loader_args = dict(shuffle=False, batch_size=64, num_workers=num_workers, pin_memory=True, collate_fn=dev_dataset.collate_fn) if cuda\
                    else dict(shuffle=False, batch_size=64)
dev_loader = DataLoader(dev_dataset, **dev_loader_args)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Check train_dataset data
cnt = 0 
for batch_idx, (data, label) in enumerate(train_dataset):
    if batch_idx < 7 and batch_idx > 3:
      print('Batch', batch_idx, ':\n', data.shape, label, '\n')
    cnt+=1
print(cnt)

In [ ]:
# Check train_loader
for batch_idx, (data, target, lenX, lenY) in enumerate(train_loader): #(data, target)
    if batch_idx < 5:
        print('Batch', batch_idx, ':\n', 'data: ', data.shape, 'target', target.shape, 'lenX', lenX, 'lenY', lenY, '\n')
        #print('Batch', batch_idx, ':\n', 'target: ', target[0], 'target.shape', target.shape)

print(len(train_loader))

Batch 0 :
 data:  torch.Size([1674, 64, 40]) target torch.Size([64, 211]) lenX tensor([1552, 1143,  391, 1037, 1511, 1523, 1576, 1355, 1123, 1421, 1639,  935,
        1569, 1448,  199,  725, 1612, 1674, 1531, 1551, 1495, 1372, 1423,  657,
        1506, 1429, 1335, 1103, 1361, 1287, 1530,  193, 1135,  970, 1581,  346,
        1292, 1075, 1303, 1350, 1603, 1251, 1457, 1641, 1507, 1559, 1328,  856,
        1582, 1376,  571, 1421, 1068,  921, 1401,  491, 1345, 1383,  506,  424,
         749,  894, 1307, 1441], dtype=torch.int32) lenY tensor([179, 150,  44, 104, 170, 167, 140, 184, 120, 131, 205, 102, 141, 132,
         17,  72, 177, 129, 128, 165, 178, 144, 140,  67, 179, 159, 134,  74,
        143, 150, 171,  14, 106,  92, 102,  33, 111, 130, 111, 156, 177, 157,
        171, 211, 135, 174, 110,  91, 180, 153,  67, 159, 129, 102, 155,  44,
        117, 147,  40,  47,  91,  69, 137, 152], dtype=torch.int32) 

Batch 1 :
 data:  torch.Size([1646, 64, 40]) target torch.Size([64, 197]) lenX ten

### Model Structure

In [ ]:
# forward function
#b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
#torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=)

In [ ]:
# Model that takes packed sequences in training
class Model(nn.Module):
    
    def __init__(self, in_size, hidden_size, nlayers, out_size, kernel_size): #(40,256,3,42,3)  
        super(Model,self).__init__()
        self.in_size=in_size
        self.hidden_size = hidden_size
        self.nlayers=nlayers
        self.rnn = nn.LSTM(input_size = in_size,hidden_size=hidden_size,num_layers=nlayers, bidirectional=True) # 1 layer, batch_size = False
        self.output = nn.Linear(hidden_size*2, hidden_size) # length of phonemes
        self.output2 = nn.Linear(hidden_size, out_size)
        self.activation = nn.ReLU()
        #self.cnn = nn.Conv1d(in_size, hidden_size, kernel_size) #(40, 256, 3)
        self.cnn = nn.Conv1d(in_size, in_size, kernel_size, padding=1) #(40, 40, 3)
        torch.nn.init.xavier_normal_(self.output.weight.data)
        torch.nn.init.xavier_normal_(self.output2.weight.data)
        torch.nn.init.xavier_normal_(self.cnn.weight.data)
    
    def forward(self, X, lengths): #X (T,B,40), lengths [T in batch]

        cnn_output = self.cnn(X.permute(1,2,0)) # CNN(B,40,T) -> output(B,40,T)
        lstm_input = cnn_output.permute(2,0,1)
        
        pack_X = nn.utils.rnn.pack_padded_sequence(lstm_input, lengths, enforce_sorted=False) # not sorted
        packed_out,_ = self.rnn(pack_X)
        out, out_lens = nn.utils.rnn.pad_packed_sequence(packed_out)

        out = self.output(out)
        out = self.activation(out)
        out2 = self.output2(out).log_softmax(2)

        return out2, out_lens

        #############################################################################################
        #out2: (T,B,40)
        #out_kens: [T in batch]




In [ ]:
# Instantiate
model = Model(40,256,5,42,3)
criterion = nn.CTCLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-3, weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, mode='max', verbose=True)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)


'''
Minimum baseline: ~25 mean Levenshtein distance within 20-30 epochs
(V)2 biLSTM (hidden size 128) + Linear
(V)optimizer Adam (weight decay 5e-6)
(V)learning rate 2e-3 (scheduled to halve every 5 epochs)
batchsize: whatever cuda allows

Ideas for improvement:
If you don't have a scheduler GET ONE! :D
larger hidden size for LSTM
more LSTM layers (num_layers)
Convolutional layers
more than 1 linear layer
dropout
tune learning rate
'''

Model(
  (rnn): LSTM(40, 256, num_layers=5, bidirectional=True)
  (output): Linear(in_features=512, out_features=256, bias=True)
  (output2): Linear(in_features=256, out_features=42, bias=True)
  (activation): ReLU()
  (cnn): Conv1d(40, 40, kernel_size=(3,), stride=(1,), padding=(1,))
)


"\nMinimum baseline: ~25 mean Levenshtein distance within 20-30 epochs\n(V)2 biLSTM (hidden size 128) + Linear\n(V)optimizer Adam (weight decay 5e-6)\n(V)learning rate 2e-3 (scheduled to halve every 5 epochs)\nbatchsize: whatever cuda allows\n\nIdeas for improvement:\nIf you don't have a scheduler GET ONE! :D\nlarger hidden size for LSTM\nmore LSTM layers (num_layers)\nConvolutional layers\nmore than 1 linear layer\ndropout\ntune learning rate\n"

### Train / Validation

In [ ]:
!pip3 install python-Levenshtein

In [ ]:
# install ctcdecode
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

fatal: destination path 'ctcdecode' already exists and is not an empty directory.
/content/drive/MyDrive/colab_notebooks/11785DL/HW3/ctcdecode
Processing /content/drive/MyDrive/colab_notebooks/11785DL/HW3/ctcdecode
  Created wheel for ctcdecode: filename=ctcdecode-1.0.2-cp37-cp37m-linux_x86_64.whl size=12878043 sha256=ad0c66c3294668e1791d597f51c1a0494a7b7fba09c1aa3bf35102b7ac761eeb
  Stored in directory: /tmp/pip-ephem-wheel-cache-l49h9_4f/wheels/56/5c/3c/32ba954f7d5d90507325b6d44ea0929643dc17ef674e329e47
Successfully built ctcdecode
  Found existing installation: ctcdecode 1.0.2
    Uninstalling ctcdecode-1.0.2:
      Successfully uninstalled ctcdecode-1.0.2
/content/drive/MyDrive/colab_notebooks/11785DL/HW3


In [ ]:
from Levenshtein import distance
from ctcdecode import CTCBeamDecoder
decoder = CTCBeamDecoder(p_map, beam_width=4, log_probs_input=True)  #phoneme map

In [ ]:
# Train the model

def train_epoch(model, train_loader, criterion, optimizer):
    
    print('Training......')
    # Set model in 'Training mode'
    model.train()

    # enumerate mini batches
    for i, (data, target, data_length, target_length) in enumerate(train_loader):
        optimizer.zero_grad() # clear the gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device

        out, out_lens = model(data, data_length) # compute the model output

        ########################################################################################################################
        #test_Y, _, _, test_Y_lens = decoder.decode(out.transpose(0,1), out_lens)

        #for j in range(len(test_Y)):
            # For the j-th sample in the batch, get the best output
        #    best_seq = test_Y[j, 0, :test_Y_lens[j, 0]]
            #print('best_seq ', best_seq)
            
            # decode model output
        #    best_pron = ''
        #    for k in best_seq:
        #        best_pron += p_map[k]
        #    print(data[j].shape, '->', best_pron)

            # decode target(answer)
        #    label_word = ''
        #    for m in target[j]:
        #        label_word += p_map[m]
        #    print('label(answer) ', '->', label_word)


            # compute the L distance (package)
        #    distance_score = distance(best_pron, label_word)
        #    print('distance_score: ', distance_score)

        ##################################################################################################################################
        loss = criterion(out, target, out_lens, target_length) # calculate CTC loss

        print('Epoch', epoch , 'Training Loss', loss.item())
        loss.backward()
        optimizer.step()

In [ ]:
# Validate the model

def val_epoch(model, dev_loader, criterion, optimizer):
    
    print('Validating......')
    # Set model in 'Validating mode'
    model.eval()

    total_distance_score = 0
    total_count = 0
    # enumerate mini batches
    for i, (data, target, data_length, target_length) in enumerate(dev_loader):
        optimizer.zero_grad() # clear the gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        out, out_lens = model(data, data_length) # compute the model output

        test_Y, _, _, test_Y_lens = decoder.decode(out.transpose(0,1), out_lens)

        #print('data.shape ', data.shape)
        #print('target.shape ', target.shape)
        #print('data_length', data_length)
        #print('target_length', target_length)
        #print('test_Y.shape', test_Y.shape)
        #print('test_Y_lens', test_Y_lens)
        #print(len(test_Y))
        #print(target)

        for j in range(len(test_Y)):
            # For the j-th sample in the batch, get the best output
            best_seq = test_Y[j, 0, :test_Y_lens[j, 0]]
            #print('best_seq ', best_seq)
            
            # decode model output
            best_pron = ''
            for k in best_seq:
                best_pron += p_map[k]
            #print(data[j].shape, '->', best_pron)

            # decode target(answer)
            label_word = ''
            for m in target[j]:
                label_word += p_map[m]
            #print('label(answer) ', '->', label_word)


            # compute the L distance (package)
            distance_score = distance(best_pron, label_word)
            #print('distance_score: ', distance_score)
            total_distance_score += distance_score
             
        total_count += len(test_Y)
        #print('score & count ', total_distance_score, total_count)

    avg_distance_score = total_distance_score / total_count
    print('avg_distance_score: ', avg_distance_score)


        
      

In [ ]:
%ls

ctcdecode/      HW3P2.gsheet     __pycache__/           test.npy
dev_labels.npy  model_results/   result3.csv            train_labels.npy
dev.npy         phoneme_list.py  sample_submission.csv  train.npy


In [ ]:
# Define number of epochs
epochs = 30
trialNumber = 5

for epoch in range(epochs):
    
    print('EEPPPPPPPOOOOOOCCCCHHHHHHH: ', epoch)
    # Train
    train_epoch(model, train_loader, criterion, optimizer)

    # Validation
    val_epoch(model, dev_loader, criterion, optimizer)
    
    # Save model
    filename = 'trial' + str(trialNumber) + 'epoch' + str(epoch) + '.pth'
    path = 'model_results/' #content/drive/MyDrive/colab_notebooks/11785DL/HW3
    torch.save(model.state_dict(), path + filename)

EEPPPPPPPOOOOOOCCCCHHHHHHH:  0
Training......


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Streaming output truncated to the last 5000 lines.
Epoch 8 Training Loss 0.5078199505805969
Epoch 8 Training Loss 0.512357771396637
Epoch 8 Training Loss 0.5181490182876587
Epoch 8 Training Loss 0.5280891060829163
Epoch 8 Training Loss 0.510024905204773
Epoch 8 Training Loss 0.5440303683280945
Epoch 8 Training Loss 0.5225182175636292
Epoch 8 Training Loss 0.5133339762687683
Epoch 8 Training Loss 0.5302037000656128
Epoch 8 Training Loss 0.5681134462356567
Epoch 8 Training Loss 0.5723434686660767
Epoch 8 Training Loss 0.514611005783081
Epoch 8 Training Loss 0.4740504324436188
Epoch 8 Training Loss 0.4909610450267792
Epoch 8 Training Loss 0.5575864315032959
Epoch 8 Training Loss 0.5181499719619751
Epoch 8 Training Loss 0.5565601587295532
Epoch 8 Training Loss 0.4938298165798187
Epoch 8 Training Loss 0.5459234714508057
Epoch 8 Training Loss 0.5344727039337158
Epoch 8 Training Loss 0.543359100818634
Epoch 8 Training Loss 0.5094870328903198
Epoch 8 Training Loss 0.49838605523109436
Epoch 8 T

In [ ]:
#import python-Levenshtein
from Levenshtein import distance

a = 'h'
b = '.DhROzizaRfrDERImyndmIhvpURHelhnhndDhfrstWrkydidWiTdEvidWhzrEnjiNflwrzlykDIzfrhdedbEbIzlithlkofin.'


edit_dist = distance(a, b)
edit_dist

97

In [ ]:
!nvidia-smi

Fri Apr  9 02:21:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    33W / 250W |  12675MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Test

In [ ]:
test_data[1].shape

(1137, 40)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X  # (timestep, band)
        self.Y = Y  # label list

        assert(len(self.X) == len(self.Y))

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, index):
        data = self.X[index]
        label = self.Y[index]
        data = torch.Tensor(data)
        label = torch.IntTensor(label)
        return data, label

    def collate_fn(self, batch):
        # 1 training data, 1 label
        batch_x = [x for x,y in batch]
        batch_y = [y for x,y in batch]
        batch_x = nn.utils.rnn.pad_sequence(batch_x)
        batch_y = nn.utils.rnn.pad_sequence(batch_y, batch_first=True)
        batch_x_len = [len(x) for x,y in batch]
        batch_y_len = [len(y) for x,y in batch]
        batch_x_len = torch.IntTensor(batch_x_len)
        batch_y_len = torch.IntTensor(batch_y_len)

        #data_length = torch.IntTensor(data_length)
        #target_length = torch.IntTensor(target_length)

        return batch_x, batch_y, batch_x_len, batch_y_len
        ###############################################################
        # batch_x: (T,B,40)     timestep(longest in batch), batch, band
        # batch_y: (B,T)        batch, labels(longest in batch)
        # batch_x_len: []       list of each timestep in batch
        # batch_y_len: []       list of each label in batch

In [ ]:
# Test data preparation
class TestDataset(Dataset):
    def __init__(self, X):
        self.X = X

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        data = self.X[index]
        data = torch.Tensor(data)
        return data

    def collate_fn(self, batch):
        batch_x = nn.utils.rnn.pad_sequence(batch)
        batch_len = [len(i) for i in batch]
        batch_len = torch.IntTensor(batch_len)

        return batch_x, batch_len

In [ ]:
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0 #change back to 8 when training for speed

#Testing
test_dataset = TestDataset(test_data)
test_loader_args = dict(shuffle=False, batch_size=64, num_workers=num_workers, pin_memory=True, collate_fn=test_dataset.collate_fn) if cuda \
                    else dict(shuffle=False, batch_size=64)
test_loader = DataLoader(test_dataset, **test_loader_args)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Check test_loader

for batch_idx, (data, data_length) in enumerate(test_loader):
    if batch_idx < 10:
        print('Batch', batch_idx, ':\n', 'data: ', data.shape, 'data_length', data_length, '\n')

print(len(test_loader))

In [ ]:
%ls

ctcdecode/      model_results/   result4.csv            train_labels.npy
dev_labels.npy  phoneme_list.py  result5.csv            train.npy
dev.npy         __pycache__/     sample_submission.csv
HW3P2.gsheet    result3.csv      test.npy


In [ ]:
# Retrieve path
filename = 'trial5epoch29.pth'
path = 'model_results/'

# Instantiate
model = Model(40,256,5,42,3)
model.load_state_dict(torch.load(path + filename))
criterion = nn.CTCLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-3, weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, mode='max', verbose=True)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

Model(
  (rnn): LSTM(40, 256, num_layers=5, bidirectional=True)
  (output): Linear(in_features=512, out_features=256, bias=True)
  (output2): Linear(in_features=256, out_features=42, bias=True)
  (activation): ReLU()
  (cnn): Conv1d(40, 40, kernel_size=(3,), stride=(1,), padding=(1,))
)


In [ ]:
def Finaltest_model(model, test_loader, criterion):
    print('Final Testing....')

    with torch.no_grad():
        model.eval()
        
        p=[]
        cnt = 0
        for batch_idx, (data, data_length) in enumerate(test_loader):
            data = data.to(device)
            out, out_lens = model(data, data_length) # compute the model output

            test_Y, _, _, test_Y_lens = decoder.decode(out.transpose(0,1), out_lens)
            for i in range(len(test_Y)):
                # For the i-th sample in the batch, get the best output
                best_seq = test_Y[i, 0, :test_Y_lens[i, 0]]
                best_pron = ''
                for k in best_seq:
                    best_pron += p_map[k]
                
                p.append(best_pron)
                cnt += 1
                print(data[i].shape, '->', best_pron)
        print(cnt)
        return p

                
p = Finaltest_model(model, test_loader, criterion)
print('Done')

Final Testing....


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([64, 40]) -> .frlhkoRdiNlIRAnthDhbOWindOAtDhfRRDrendhvDhHol.AndlukthloNDhvisthvhgaRdhnWok.kaphthd.WiTklOslISEvingRAs.hndbodrdWiTshmRUdAndimbItoRhtemptAtRhvrI.
torch.Size([64, 40]) -> .Dhpwdh.vEpRzkhntensthnhWotr.AndDIeR.putinhvylhntAkShnthshplyDhvAkim.leftbyDhkandhnsEShn.hvDhmist.RwzizitselfintUhWrldWed.
torch.Size([64, 40]) -> .AndfiRstDwbikhzyfAniS.AnAmsInUmoR.
torch.Size([64, 40]) -> .fAdhm.stiksfIt.
torch.Size([64, 40]) -> .DhveRIhbilhtIhvmoltfhlpaRt.
torch.Size([64, 40]) -> .AtDhHedhvDhTiNkIzWrgitghsezhl.hndbhhmbRyd.HUHAdDhpeRhnnHizSOdr.hndDeRWrshpoRtidbykApthnkalI.AndkApthntentUhnt.AndROzhlIDhWic.
torch.Size([64, 40]) -> .yWhzHOpiNDhtifDhpEpryWicHIROtWhzTinshmtREshvitmytkhmTRUhpanDispaliSsrfhs.nyysInhTiN.
torch.Size([64, 40]) -> .RabithnDhlithltamplrbitWInDhmtRydthfoRsDhskWyrdhstAndbAk.hndveRIvAlIilItiDIeztUkhmpoRtDhmselvz.
torch.Size([64, 40]) -> .DIindIhnaolsOTotnhTiNhv.
torch.Size([64, 40]) -> .sUllIOhzDIhtrEShnhvfRAns.hndHrtAhntWhzDhRIlshmpoRt.hvolDhkamhtIz.WicDhgR

In [ ]:
len(p)

2561

In [ ]:
# Final work!
import pandas as pd

result = pd.DataFrame(p)
result = result.reset_index()
#result.rename(columns={'index': 'id', '0': 'lable'}, inplace=True)
result.columns =['id', 'label'] 
result.to_csv('result9.csv', index=False)

### Kaggle Submission

In [ ]:
%ls

ctcdecode/      model_results/   result4.csv  result8.csv            train.npy
dev_labels.npy  phoneme_list.py  result5.csv  sample_submission.csv
dev.npy         __pycache__/     result6.csv  test.npy
HW3P2.gsheet    result3.csv      result7.csv  train_labels.npy


In [ ]:
%cd /content/drive/MyDrive/colab_notebooks/11785DL/HW3/

/content/drive/MyDrive/colab_notebooks/11785DL/HW3


In [ ]:
# Use Kaggle API to submit your result
!kaggle competitions submit -c 11785-spring2021-hw3p2 -f result9.csv -m "9th submission"

100% 204k/204k [00:03<00:00, 62.2kB/s]
Successfully submitted to 11785 Homework 3 Part 2: Seq to Seq